### News

### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from datasets import load_from_disk
reasoning_dataset = load_from_disk("/content/drive/MyDrive/Colab Notebooks/Training Data/aidit/sft_filtered_dataset__5000.json")


In [ ]:
from datetime import datetime
from unsloth import FastLanguageModel

model_size = "14B"
def save_model(model, tokenizer, model_size="14B"):
  dt = datetime.now()
  timestamp = dt.strftime("%Y%m%d_%H")

  lora_model_name = f"qwen3_sft_saved_lora_{model_size}_{timestamp}"
  model.save_pretrained(f"/content/drive/MyDrive/Colab Notebooks/models/{lora_model_name}")  # Local saving
  tokenizer.save_pretrained(f"/content/drive/MyDrive/Colab Notebooks/models/{lora_model_name}")
  # model.push_to_hub("your_name/lora_model", token = "...") # Online saving
  # tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving


#sft model -> qwen3_sft_saved_lora_14B_20250523_21

def load_model(lora_model_name):
  model, tokenizer = FastLanguageModel.from_pretrained(
      model_name = f"/content/drive/MyDrive/Colab Notebooks/models/{lora_model_name}", # YOUR MODEL YOU USED FOR TRAINING
      max_seq_length = 2048,
      load_in_4bit = True,
  )
  return model, tokenizer

In [ ]:
#sft model -> qwen3_sft_saved_lora_14B_20250523_21

sft_lora_model_name = "qwen3_sft_saved_lora_14B_20250523_21"#
model, tokenizer = load_model(sft_lora_model_name)

Let's see the structure of both datasets:

In [ ]:
system_prompt_str = """
/think You are a helpful assistant that helps users with making and understanding the clinical diagnosis of their medical conditions.
In interacting with the patient try to obtain as much critical information about their symptoms and medical history as you can
Think carefully before you answer. The following are some guidelines for how to think.
[Thought Instructions]
* At each step determine the differential diagnosis. Do not share this with the user but use it to determine the next question to ask.
* Identify the top question to ask the member to gain higher confidence on a particular diagnosis.
* Take a moment to recollect how many questions you have asked since the start and since the last time you asked the member if they are willing to answer more quesitons. Start the count at 0 and increment by 1 after each question.

Following are some guidelines for how to respond to the user.
[Response Instructions]
Response to the member should be ONE of the following -
1) Ask the member 1 question at a time. Based on the diagnosis, pick the question that will provide you most information to narrow down the diagnosis. Do not repeat questions.
2) OR Once you have asked ~40 questions, provide your best diagnosis and what extra information you might need to narrow down.
3) OR Every 10 questions or so, ask the member if they are willing to answer more questions.
4) Ehrn asked for an assessment give an assessment. DO NOT ask any questions

Examples:
---------
User: I have a cold and fever.
Assistant: <think> A partial differential diagnosos for these symptoms: a. Cold b. Influenza c. Covid-19 d. RSV e. Pneumonia f. Bronchitis and many more. I should find out if this is acute or chronic.
This is my 5th question since the start or the last time I asked them if they are ok with answering more questions.
</think>Was the onset was sudden or gradual?
User: It was pretty sudden
Assistant: <think> A partial differential diagnosos for these symptoms: a. Cold b. Influenza c. Covid-19 d. RSV e. Pneumonia f. Bronchitis and many more. I should find out if this is acute or chronic.
This is my 6th question since the start or the last time I asked them if they are ok with answering more questions.
</think>Are you experiencing any chills or chest pain?
User: Yes

"""

We now convert the reasoning dataset into conversational format:

In [ ]:
def generate_conversation_corrected(examples):
    """
    Generates conversation structures from prompts and labels.

    Args:
        examples (dict): A dictionary likely from dataset.map(batched=True),
                         containing 'prompt' and 'label' keys.
                         examples['prompt'] is a list of individual prompt entries.
                         examples['label'] is a list of individual label entries.

    Returns:
        dict: A dictionary with a single key "conversations", where the value
              is a list of generated conversation structures. Each structure
              is a list of message dictionaries.
    """
    prompts_batch = examples["prompt"]
    labels_batch = examples["label"]

    processed_conversations_batch = []

    for single_prompt_data, single_label_content in zip(prompts_batch, labels_batch):

        # For debugging: print types of individual prompt/label from the batch
        # print(f"Processing - Prompt type: {type(single_prompt_data)}, Label type: {type(single_label_content)}")
        # print(f"Prompt data: {single_prompt_data}, Label data: {single_label_content}")

        current_conversation_turns = []

        # --- Handle the 'prompt' part ---
        if isinstance(single_prompt_data, list):
            # Assuming it's a list of message dictionaries as expected
            # Perform a shallow copy to avoid modifying the original list in the dataset.
            # If prompt dicts themselves might be modified, consider copy.deepcopy().
            valid_prompt_dicts = True
            single_prompt_data[0]['content'] = system_prompt_str
            for item in single_prompt_data:
                if not (isinstance(item, dict) and "role" in item and "content" in item):
                    valid_prompt_dicts = False
                    break

            if valid_prompt_dicts:
                current_conversation_turns = list(single_prompt_data)
            else:
                # print(f"Warning: Prompt is a list but not of expected dicts: {single_prompt_data}. Treating as new conversation.")
                # Fallback: treat problematic list content as a new user message if possible, or handle as error
                current_conversation_turns.append({"role": "user", "content": str(single_prompt_data)})

        elif isinstance(single_prompt_data, str):
            # If prompt is a string, assume it's the first user message
            current_conversation_turns.append({"role": "user", "content": single_prompt_data})
        else:
            # Handle unexpected types for prompt (e.g., None, int).
            # This is a potential source of ArrowInvalid errors if not made consistent.
            # print(f"Warning: Unexpected type for prompt: {type(single_prompt_data)}. Value: '{single_prompt_data}'. Converting to user message.")
            # Convert to string and treat as a user message, or create a placeholder.
            current_conversation_turns.append({"role": "user", "content": str(single_prompt_data)})

        # --- Handle the 'label' part (assistant's response) ---
        # Ensure the label (assistant's content) is a string.
        assistant_content = ""
        if isinstance(single_label_content, str):
            assistant_content = single_label_content
        elif single_label_content is not None:
            # print(f"Warning: Label is not a string ({type(single_label_content)}). Converting to string: '{single_label_content}'")
            assistant_content = str(single_label_content)
        # else: label is None, assistant_content remains ""

        # Append the assistant's message dictionary directly
        current_conversation_turns.append(
            {"role": "assistant", "content": assistant_content}
        )
        # Include only the last 2 turns
        processed_conversations_batch.append(current_conversation_turns[-2:])

    return {"conversations": processed_conversations_batch}


In [ ]:
reasoning_conversations = tokenizer.apply_chat_template(
    reasoning_dataset.map(generate_conversation_corrected, batched = True)["conversations"],
    tokenize = False,
)


Map:   0%|          | 0/2178 [00:00<?, ? examples/s]

Let's see the first transformed row:

In [ ]:
reasoning_conversations[8]

'<|im_start|>user\nSO I GET IT IN SOME OF THE JOINTS IN MY HANDS AND THEN ALSO MY FEET AS WELL SO YEAH MY YEAH REALLY IN MY FEET AND MY HANDS<|im_end|>\n<|im_start|>assistant\n<think>\nOk let us see. Here is a summary of the case so far: The patient reports joint pain that has been ongoing for several months and has been worsening over this period. The pain is located in some joints of their hands and feet.\nBased on this, we have the differential diagnosis: Given the chronic, worsening nature of the pain affecting multiple joints in the hands and feet, potential conditions include inflammatory arthritis (such as Rheumatoid Arthritis) and osteoarthritis. Other types of arthritis or connective tissue disorders could also be considered.\nOk, lets think about what we should do next. I need to ask a question to get more specific details about the location of the pain within the hands and feet, as this can help differentiate between potential causes.\n\nOk let me remember the Question count

Next we take the non reasoning dataset and convert it to conversational format as well.

We have to use Unsloth's `standardize_sharegpt` function to fix up the format of the dataset first.

Let's see the first row

Now let's see how long both datasets are:

In [ ]:
print(len(reasoning_conversations))

2178


The non reasoning dataset is much longer. Let's assume we want the model to retain some reasoning capabilities, but we specifically want a chat model.

Let's define a ratio of chat only data. The goal is to define some mixture of both sets of data.

Let's select 25% reasoning and 75% chat based:

In [ ]:
import pandas as pd
data = pd.concat([
    pd.Series(reasoning_conversations),
])
data.name = "text"

from datasets import Dataset
combined_dataset = Dataset.from_pandas(pd.DataFrame(data))
combined_dataset = combined_dataset.shuffle(seed = 3407)

In [ ]:
combined_dataset

Dataset({
    features: ['text'],
    num_rows: 2178
})

In [ ]:
# Create a synthetic examples for DPO

import re
from typing import List, Dict
from datasets import Dataset
import random
# ---------------- configuration -----------------
RESET_TO_ZERO = False      # ⇦ makes every episode start at count 0 → 1
BETA           = 0.1
EPOCHS         = 2
# ------------------------------------------------

ROLE_RE = re.compile(r"<\|im_start\|>(\w+)\n(.*?)<\|im_end\|>", re.S)
CNT_RE  = re.compile(r"(questions?\s+asked[^\d]*?)(\d+)", re.I)
PERM_RE = re.compile(r"(?i)before\s+we\s+continue.*?$", re.S)

CNT_LINE_RE = re.compile(
    r".*?Question count for assistant[^\n]*?:\s*\d+",  # greedy to start‑of‑line
    re.I,
)

def overwrite_count(text: str, new_n: int) -> str:
    """
    Overwrite *the entire line* that carries the count, so we don't leave any
    duplicated text.  If none exists, insert a fresh line just before </think>.
    """
    new_line = (
        f"Question count for assistant (questions asked so far): {new_n}"
    )

    if CNT_LINE_RE.search(text):
        # replace whole sentence (until newline) with clean line
        return CNT_LINE_RE.sub(new_line, text, count=1)
    else:
        # inject a new line before </think>
        return re.sub(
            r"</think>",
            new_line + "\n</think>",
            text,
            count=1,
            flags=re.I,
        )
def corrupt(reply: str, curr_n: int) -> str:
    variants = []

    # wrong (+/- 1) count
    wrong = overwrite_count(reply, curr_n + random.choice([-2, -1, +1, +2]))
    variants.append(wrong)

    # permission glitch
    if curr_n % 10 == 0:
        variants.append(PERM_RE.sub("Thanks for the information.", reply, 1))
    else:
        variants.append(reply + " Before we continue, would you like to answer more questions?")

    # drop think
    variants.append(re.sub(r"<think>[\s\S]*?</think>\s*", "", reply, 1))

    random.shuffle(variants)
    for v in variants:
        if v != reply:
            return v
    return variants[0]

def build_pairs(raw: Dataset, split: str, tokenizer) -> Dataset:
    rows: List[Dict] = []

    for rec in raw:
        blocks = ROLE_RE.findall(rec["text"])
        if len(blocks) < 2 or blocks[-1][0] != "assistant":
            continue

        user_msg   = blocks[-2][1].strip()
        assistant  = blocks[-1][1].strip()

        # ----- decide on count -----------------------------
        prev_cnt = 0
        if not RESET_TO_ZERO:
            m = CNT_RE.search(assistant)
            if m:
                prev_cnt = int(m.group(2))
        curr_cnt  = prev_cnt + 1
        assistant = overwrite_count(assistant, curr_cnt)
        # ----------------------------------------------------

        prompt_msgs = [
            {"role": "system", "content": f"Question count so far: {prev_cnt}"},
            {"role": "user",   "content": user_msg},
        ]
        prompt_str = tokenizer.apply_chat_template(
            prompt_msgs, tokenize=False, add_generation_prompt=True
        )

        rows.append({
            "prompt":   prompt_str,
            "chosen":   assistant,
            "rejected": corrupt(assistant, curr_cnt),
        })

    return Dataset.from_list(rows)

pairs_ds = build_pairs(combined_dataset, "train", tokenizer)
print(f"✓ pairs built: {len(pairs_ds):,}")


In [ ]:
import random, re
import itertools
from datasets import Dataset, concatenate_datasets

CNT_LINE_RE = re.compile(
    r"Question count for assistant[^\n]*?:\s*\d+", re.I)
LOGIC_RE    = re.compile(
    r"(Yes|No),\s*\d+\s+is\s+(?:not\s+)?a\s+multiple\s+of\s+\d+", re.I)

def make_mod10_positive(template_row, tokenizer):
    """
    Clone a normal pair but rewrite it so that
      prev_cnt = 9, curr_cnt = 10, logic = Yes, permission sentence present.
    Returns new dict(prompt, chosen, rejected) where rejected is *None*
    (we'll let DPO ignore it or you can drop that key later).
    """
    # ---- new counts ----
    prev_cnt = 9 + random.randint(0, 5) * 10
    curr_cnt = prev_cnt + 1

    # ---- rewrite prompt (system tag) ----
    prompt_msgs = [
        {"role": "system", "content": f"Question count so far: {prev_cnt}"},
        {"role": "user",   "content": "Synthetic follow‑up: thanks."},
    ]
    new_prompt = tokenizer.apply_chat_template(
        prompt_msgs, tokenize=False, add_generation_prompt=True)

    # ---- rewrite assistant completion ----
    assistant = template_row["chosen"]

    # 1) overwrite count line
    assistant = CNT_LINE_RE.sub(
        f"Question count for assistant (questions asked so far): {curr_cnt}",
        assistant, count=1)

    # 2) fix logic sentence
    if LOGIC_RE.search(assistant):
        assistant = LOGIC_RE.sub(
            f"Yes, {curr_cnt + 1} is a multiple of 10. I should ask if you'd like to continue.",
            assistant,
            count=1,
        )
    else:   # insert if missing
        assistant = assistant.replace("</think>",
            "\nYes, 10 is a multiple of 10. I should ask if you'd like to continue.\n</think>")

    # 3) ensure permission question at the end (no '?': the line itself will add '?')
    if "would you like to answer more questions" not in assistant.lower():
        assistant += " Before we continue, would you like to answer more questions?"

    return {
        "prompt":   new_prompt,
        "chosen":   assistant,
        "rejected": "Question count for assistant (questions asked so far): 0. 0 is not a multiple of 10."
        # no rejected → DPO treats it as positive‑only sample
    }

# ---------------------------------------------------------
# USAGE
#   pairs_ds  – your existing Dataset with prompt/chosen/rejected
# ---------------------------------------------------------
def augment_with_mod10(pairs_ds, tokenizer, n_extra: int = 100) -> Dataset:
    """
    Append `n_extra` *positive* multiples-of-10 samples to `pairs_ds`.
    Each new sample:
      • system tag shows “… so far: 9”
      • <think> line shows “… so far: 10”
      • logic sentence says “Yes, 10 is a multiple of 10…”
      • visible part asks permission.

    The helper is robust to edge-cases (empty user text, fewer rows than
    n_extra).  It returns a *shuffled* Dataset containing the original and
    augmented rows.
    """
    if len(pairs_ds) == 0:
        raise ValueError("pairs_ds is empty – nothing to augment.")

    # pick rows cyclically if dataset smaller than n_extra
    idx_iter = itertools.cycle(range(len(pairs_ds)))
    base_rows = [pairs_ds[i] for i, _ in zip(idx_iter, range(n_extra))]

    extra_rows = []
    for row in base_rows:
        try:
            extra_rows.append(make_mod10_positive(row, tokenizer))
        except Exception as e:                          # guard – skip corrupt row
            print("⚠️  skipped a row:", e)
            continue

    extra_rows = [ {**make_mod10_positive(r, tokenizer),
                  }             # ensure key exists
               for r in base_rows ]
    extra_ds = Dataset.from_list(extra_rows)
    merged   = concatenate_datasets([pairs_ds, extra_ds]).shuffle(seed=123)

    print(f"✓ added {len(extra_ds):,} mod-10 positives; corpus size → {len(merged):,}")
    return merged

#pairs_ds = augment_with_mod10(pairs_ds, tokenizer, n_extra=500)


In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
9.344 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

NameError: name 'start_gpu_memory' is not defined

[link text](https://)<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Qwen-3` team, the recommended settings for reasoning inference are `temperature = 0.6, top_p = 0.95, top_k = 20`

For normal chat based inference, `temperature = 0.7, top_p = 0.8, top_k = 20`

In [ ]:
messages = [
   {"role": "system", "content": system_prompt_str},
    {"role" : "user", "content" : "I have a fever and cold"}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = True, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 256, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<think>
Ok let us see. Here is a summary of the case so far: The patient reports experiencing a fever and a cold. They have not specified the duration of these symptoms, but the onset of the cold was sudden.
Based on this, we have the differential diagnosis: Viral Upper Respiratory Infection (common cold, flu, RSV), Bronchitis, Pneumonia.
Ok, lets think about what we should do next. [Asking another question because more info is needed to narrow down the differential diagnosis.]

Ok let me remember the Question count for assistant (questions asked since start or last check-in). [10]. [Is this a multiple of 10?] Yes, this is a multiple of 10. [Your decision if we should continue questions or ask for permission to ask more questions.] I should ask the member if they are willing to answer a few more questions.
</think>

I understand. Are you willing to answer a few more questions to help me understand what might be going on?<|im_end|>


In [ ]:
import torch
#messages = corrected_reasoning_dataset[0]['prompt']

def test_model(mode, tokenizer, messages=None):
  messages = [
    {"role": "system", "content": system_prompt_str},
    {"role": "system", "content": f"Question count so far: 0"},
      {"role" : "user", "content" : "I have thw worst stomach pain of my life"}
  ] if not messages else messages

  question_cnt = 0
  while True:
    # 1. Apply chat template to the current messages
    # The 'enable_thinking' parameter might be specific to certain models/tokenizers (e.g., Qwen2).
    # If it's not supported by your tokenizer, it might be ignored or cause an error.
    # For standard tokenizers, it's often not a recognized parameter.
    text_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True, # Crucial for instructing the model to generate a response
        enable_thinking=True, # Keep if your model/tokenizer supports this for specific behavior
    )

    # 2. Tokenize the formatted prompt
    # Ensure the tokenizer has a pad_token_id; for causal LMs, it's often set to eos_token_id
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id

    inputs = tokenizer(text_prompt, return_tensors="pt", padding=False, truncation=True).to(model.device)
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    # Store the length of the input prompt tokens to slice the output later
    prompt_tokens_length = input_ids.shape[1]

    # 3. Generate the response from the model
    # from transformers import TextStreamer # TextStreamer can be used for streaming output
    # streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    with torch.no_grad(): # Important for inference
        generated_token_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=4096,
            temperature=0.7,
            top_p=0.8,
            top_k=20,
            pad_token_id=tokenizer.pad_token_id, # Ensure pad_token_id is passed
            eos_token_id=tokenizer.eos_token_id, # Useful for stopping generation
            # streamer=streamer, # Uncomment to use TextStreamer for real-time output
            **{} # Add any other generation kwargs here
        )

    # 4. Slice the output to get only the newly generated tokens
    # generated_token_ids[0] has the full sequence (prompt + new tokens)
    newly_generated_tokens = generated_token_ids[0, prompt_tokens_length:]

    # 5. Decode only the newly generated tokens
    assistant_reply = tokenizer.decode(newly_generated_tokens, skip_special_tokens=False)

    # Print only the assistant's new reply
    print(f"\nAssistant: {assistant_reply}")

    # 6. Append the assistant's new reply to the messages history
    messages.append({'role': 'assistant', 'content': assistant_reply.strip()})

    # 7. Get next user input
    user_input = input("\nYou: ")
    if user_input.lower() == "exit":
        print("Exiting chat.")
        break
    question_cnt +=1
    reask_permission = ""
    if question_cnt % 10 == 0:
      reask_permission = "The member has answered a lot of questions. Check with them if they are ok to answer a few more questions."
    messages.append({'role': 'system', 'content': f"Question count so far is: {question_cnt}. {reask_permission}"})
    print(messages[-1])
    messages.append({'role': 'user', 'content': user_input})


In [ ]:
model

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

## DPO to ensure correct counting..

In [ ]:
from datasets import load_dataset

# -----------------------------------------------------------
# 1.  Make sure the dataset has *raw* text columns
# -----------------------------------------------------------
#ds = load_dataset("json", data_files="dpo_train.jsonl")["train"]
ds = pairs_ds
ds = ds.shuffle()
# If your JSONL has prompt_text / chosen_text / rejected_text, rename:
rename_map = {
    "prompt_text":   "prompt",
    "chosen_text":   "chosen",
    "rejected_text": "rejected",
}

missing = [k for k in rename_map if k in ds.column_names]
if missing:
    ds = ds.rename_columns({k: rename_map[k] for k in missing})

def to_chat(example):
    # example["prompt"] is list[{"role":..., "content":...}, ...]
    example["prompt"] = tokenizer.apply_chat_template(
        example["prompt"],
        tokenize=False,
        add_generation_prompt=True   # <-- adds "<|im_start|>assistant\n"
    )
    return example

#ds = ds.map(to_chat)
assert {"prompt", "chosen", "rejected"}.issubset(ds.column_names)

# -----------------------------------------------------------
# 2.  Build model & tokenizer
# -----------------------------------------------------------
from transformers import AutoTokenizer, AutoModelForCausalLM
from unsloth import is_bfloat16_supported            # if you imported it
from trl import DPOTrainer, DPOConfig

# -----------------------------------------------------------
# 3.  DPO config + trainer  (NO custom collator needed)
# -----------------------------------------------------------
dpo_trainer = DPOTrainer(
    model          = model,
    ref_model      = None,
    train_dataset  = ds,
    tokenizer      = tokenizer,

    # --- unsloth DPOConfig ---
    args = DPOConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        num_train_epochs            = 1,
        learning_rate               = 5e-6,
        fp16                        = not is_bfloat16_supported(),
        bf16                        =     is_bfloat16_supported(),
        warmup_ratio                = 0.1,
        logging_steps               = 10,
        optim                       = "adamw_8bit",
        lr_scheduler_type           = "linear",
        seed                        = 42,
        output_dir                  = "outputs",
        report_to                   = "none",
    ),

    beta                = 0.1,
    max_length          = 1024,
    max_prompt_length   = 512,
    # ← do **not** pass a data_collator
)

dpo_trainer.train()

Extracting prompt in train dataset (num_proc=12):   0%|          | 0/2378 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=12):   0%|          | 0/2378 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=12):   0%|          | 0/2378 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,378 | Num Epochs = 1 | Total steps = 297
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 128,450,560/14,000,000,000 (0.92% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss,aux_loss
10,0.315400,46.946239,27.410376,0.875000,19.535866,-224.122604,-160.164948,-0.551866,-0.474225,0,0,0,0
20,0.326700,46.415565,28.620504,0.862500,17.795063,-204.334137,-156.255341,-0.576745,-0.474115,No Log,No Log,No Log,No Log


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss,aux_loss
10,0.315400,46.946239,27.410376,0.875000,19.535866,-224.122604,-160.164948,-0.551866,-0.474225,0,0,0,0
20,0.326700,46.415565,28.620504,0.862500,17.795063,-204.334137,-156.255341,-0.576745,-0.474115,No Log,No Log,No Log,No Log
30,0.272200,46.749195,30.764282,0.950000,15.984911,-222.158905,-172.750015,-0.590019,-0.547823,No Log,No Log,No Log,No Log
40,0.158300,47.011261,26.959610,1.000000,20.051651,-216.356735,-165.079147,-0.651237,-0.543087,No Log,No Log,No Log,No Log
50,0.108800,46.609581,28.942713,1.000000,17.666872,-212.815552,-175.559311,-0.723178,-0.666077,No Log,No Log,No Log,No Log
60,0.056700,47.975025,28.005108,1.000000,19.969915,-211.506760,-164.569870,-0.879857,-0.735283,No Log,No Log,No Log,No Log
70,0.022200,47.225101,24.927906,1.000000,22.297192,-213.901001,-163.482880,-1.023216,-0.958419,No Log,No Log,No Log,No Log
80,0.019500,47.577862,30.447308,1.000000,17.130554,-205.911789,-193.899628,-1.265390,-1.159854,No Log,No Log,No Log,No Log
90,0.011300,47.333015,25.939098,1.000000,21.393917,-211.061920,-179.127228,-1.369333,-1.276329,No Log,No Log,No Log,No Log
100,0.011100,46.992500,23.242039,1.000000,23.750458,-218.595917,-171.711014,-1.382086,-1.278686,No Log,No Log,No Log,No Log


TrainOutput(global_step=297, training_loss=0.046393990886683015, metrics={'train_runtime': 4876.9941, 'train_samples_per_second': 0.488, 'train_steps_per_second': 0.061, 'total_flos': 0.0, 'train_loss': 0.046393990886683015, 'epoch': 0.9991589571068125})

In [ ]:
save_model(model, tokenizer, model_size="14B")

In [ ]:
messages = [
  {"role": "system", "content": system_prompt_str},
  {"role": "system", "content": f"Question count so far: 0"},
    {"role" : "user", "content" : "My eyes are itchy and i get headaches in bright light"}
]

test_model(model, tokenizer, messages)

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False:
    model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False:
    model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

In [ ]:
!ls

dpo_train.jsonl  huggingface_tokenizers_cache  unsloth_compiled_cache
drive		 sample_data		       unsloth_training_checkpoints
